In [17]:
# Install necessary libraries
#%pip install numpy
#%pip install pandas
#%pip install statsmodels
#%pip install scikit-learn
#%pip install joblib

     ---------------------------------------- 9.0/9.0 MB 17.4 MB/s eta 0:00:00
     ---------------------------------------- 233.8/233.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# Load packages
import json
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
import joblib

In [19]:
#Load dataset
df = pd.read_csv("C:\\Users\\Tan Pheng Chiang\\Downloads\\Sales_Data.csv", sep = ',')
# Filter incorrect Quantity values
df = df[df.Quantity > 0]
df.Date = pd.to_datetime(df.Date, format='%d-%m-%Y')
# Get 2 years data only
df = df[(df.Date >= '2021-01-01') & (df.Date <= '2022-12-31')]
#Anonymize customer and product info
df.Customer = pd.factorize(df.Customer)[0]
df.Product = pd.factorize(df.Product)[0]
# Drop unnecessary column
df = df.drop(['Amount'], axis=1)
df.head()

,Date,Customer,Product,Quantity
2070078,2021-01-01,0,0,3.0
2070079,2021-01-01,0,1,3.0
2070080,2021-01-01,0,2,3.0
2070081,2021-01-01,0,3,2.0
2070082,2021-01-01,0,4,3.0


In [20]:
# Get the earliest and latest dates in the dataset
earliest_date = df.Date.min()
latest_date = df.Date.max()

# Create a list of dates strings between the earliest and latest dates in the sales data
date_list = pd.date_range(earliest_date, latest_date).unique()

# Create a new DataFrame with all possible combinations of product, customer and date
product_list = df.Product.unique()
customer_list = df.Customer.unique()
all_combinations = pd.MultiIndex.from_product([product_list, customer_list, date_list], names=['Product', 'Customer', 'Date'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

# Merge the sales data with the all_combinations_df to include all possible combinations
merged_df = all_combinations_df.merge(df, on=['Product', 'Customer', 'Date'], how='left')

# Fill any missing sales quantity with zero
merged_df.Quantity = merged_df.Quantity.fillna(0)
merged_df.head()

,Product,Customer,Date,Quantity
0,0,0,2021-01-01,3.0
1,0,0,2021-01-01,4.0
2,0,0,2021-01-01,4.0
3,0,0,2021-01-01,2.0
4,0,0,2021-01-01,2.0


In [21]:
# Flatten dataframe into multi-index columns Product and Customer
sales_df = merged_df.pivot_table(index='Date', columns=['Product', 'Customer'], values='Quantity', aggfunc='sum')
sales_df.head()

Product       0                                                        ...   
Customer      0       1     2    3    4     5    6     7     8    9    ...   
Date                                                                   ...   
2021-01-01  128.0  4224.0  10.0  5.0  2.0  10.0  2.0  51.0   2.0  2.0  ...  \
2021-01-02  145.0  5444.0   0.0  0.0  0.0   0.0  0.0  40.0   0.0  3.0  ...   
2021-01-03  101.0  4152.0   0.0  0.0  0.0   0.0  0.0  43.0   3.0  2.0  ...   
2021-01-04  196.0  5507.0   0.0  0.0  0.0   0.0  0.0  66.0   0.0  2.0  ...   
2021-01-05  180.0  6527.0  10.0  5.0  4.0  11.0  8.0  51.0  14.0  4.0  ...   

Product     192                                               
Customer    164  165  166  167  168  169  170  171  172  173  
Date                                                          
2021-01-01  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-02  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-03  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-04  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2021-01-05  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 33582 columns]

In [23]:
# min-max normalization along date axis
minimum = np.min(sales_df, axis=0)
maximum = np.max(sales_df, axis=0)

# Normalize the matrix using min-max normalization
sales_df_normalized = (sales_df - minimum) / (maximum - minimum)
sales_df_normalized[np.isnan(sales_df_normalized)] = 0
sales_df_normalized.head()

Product          0                                                             
Customer         0         1         2         3         4     5         6     
Date                                                                           
2021-01-01  0.479401  0.543420  0.588235  0.333333  0.166667  0.50  0.068966  \
2021-01-02  0.543071  0.700373  0.000000  0.000000  0.000000  0.00  0.000000   
2021-01-03  0.378277  0.534157  0.000000  0.000000  0.000000  0.00  0.000000   
2021-01-04  0.734082  0.708478  0.000000  0.000000  0.000000  0.00  0.000000   
2021-01-05  0.674157  0.839702  0.588235  0.333333  0.333333  0.55  0.275862   

Product                                ...  192                                 
Customer         7         8      9    ...  164  165  166  167  168  169  170   
Date                                   ...                                      
2021-01-01  0.467890  0.090909  0.050  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  \
2021-01-02  0.366972  0.000000  0.075  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-01-03  0.394495  0.136364  0.050  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-01-04  0.605505  0.000000  0.050  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-01-05  0.467890  0.636364  0.100  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Product                    
Customer    171  172  173  
Date                       
2021-01-01  0.0  0.0  0.0  
2021-01-02  0.0  0.0  0.0  
2021-01-03  0.0  0.0  0.0  
2021-01-04  0.0  0.0  0.0  
2021-01-05  0.0  0.0  0.0  

[5 rows x 33582 columns]

In [25]:
#80-20 train-test split chronologically
top_80_percent_dates = int(sales_df_normalized.shape[0] * 0.8)
last_20_percent_dates = int(sales_df_normalized.shape[0]) - top_80_percent_dates

train_df = sales_df_normalized.head(top_80_percent_dates)
test_df = sales_df_normalized.tail(last_20_percent_dates)
train_df.head()

Product          0                                                             
Customer         0         1         2         3         4     5         6     
Date                                                                           
2021-01-01  0.479401  0.543420  0.588235  0.333333  0.166667  0.50  0.068966  \
2021-01-02  0.543071  0.700373  0.000000  0.000000  0.000000  0.00  0.000000   
2021-01-03  0.378277  0.534157  0.000000  0.000000  0.000000  0.00  0.000000   
2021-01-04  0.734082  0.708478  0.000000  0.000000  0.000000  0.00  0.000000   
2021-01-05  0.674157  0.839702  0.588235  0.333333  0.333333  0.55  0.275862   

Product                                ...  192                                 
Customer         7         8      9    ...  164  165  166  167  168  169  170   
Date                                   ...                                      
2021-01-01  0.467890  0.090909  0.050  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  \
2021-01-02  0.366972  0.000000  0.075  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-01-03  0.394495  0.136364  0.050  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-01-04  0.605505  0.000000  0.050  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-01-05  0.467890  0.636364  0.100  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

Product                    
Customer    171  172  173  
Date                       
2021-01-01  0.0  0.0  0.0  
2021-01-02  0.0  0.0  0.0  
2021-01-03  0.0  0.0  0.0  
2021-01-04  0.0  0.0  0.0  
2021-01-05  0.0  0.0  0.0  

[5 rows x 33582 columns]

In [28]:
# Dimensionality reduction using PCA
pca = PCA(n_components = 0.95)
pca.fit(train_df)
train_df_reduced = pca.transform(train_df)
train_df_reduced.shape

(584, 262)

In [29]:
#adfuller test to test stationary
def adfuller_test(series, signif=0.05, index='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)
    if p_value <= signif:
        # Series is stationary
        return True
    else:
        # Series is not stationary
        return False

In [31]:
for index in range(train_df_reduced.shape[1]):
    if not adfuller_test(train_df_reduced[:, index], index=index):
        print(f"Index {index} is not stationary")

Index 0 is not stationary
Index 2 is not stationary
Index 3 is not stationary
Index 5 is not stationary
Index 10 is not stationary
Index 12 is not stationary
Index 13 is not stationary
Index 16 is not stationary
Index 17 is not stationary
Index 18 is not stationary
Index 19 is not stationary
Index 20 is not stationary
Index 21 is not stationary
Index 22 is not stationary
Index 24 is not stationary
Index 26 is not stationary
Index 32 is not stationary
Index 44 is not stationary


In [32]:
# Differentiate train_df_reduced until stationary is achieved
difference_count = 0
all_stationary = False
train_df_reduced_diff = train_df_reduced
while not all_stationary:
    difference_count += 1
    train_df_reduced_diff = np.diff(train_df_reduced_diff, axis=0)
    all_stationary = True
    for index in range(train_df_reduced_diff.shape[1]):
        if not adfuller_test(train_df_reduced_diff[:, index], index=index):
            all_stationary = False

print(f'Number of differencing required: {difference_count}')

Number of differencing required: 1


In [33]:
# Initialize VAR model
model = VAR(train_df_reduced_diff)
# Select best number of lags based on AIC
order_selection = model.select_order(maxlags=1)
order_selection.summary()

,AIC,BIC,FPE,HQIC
0,-564.2,-562.2*,9.767e-246,-563.4
1,-904.1*,-387.1,0.000*,-702.6*


In [34]:
# Lag 1 has the best AIC score
var = model.fit(1)

In [36]:
# save preprocessing objects and VAR model
joblib.dump(train_df_reduced, "./train_df_reduced.joblib", compress=True)
joblib.dump(train_df_reduced_diff, "./train_df_reduced_diff.joblib", compress=True)
joblib.dump(maximum, "./maximum.joblib", compress=True)
joblib.dump(minimum, "./minimum.joblib", compress=True)
joblib.dump(product_list, "./product_list.joblib", compress=True)
joblib.dump(customer_list, "./customer_list.joblib", compress=True)
joblib.dump(pca, "./pca.joblib", compress=True)
joblib.dump(var, "./var.joblib", compress=True)

['./var.joblib']